# Vector DB Question Answering

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.indexes.vectorstore import VectorstoreIndexCreator

In [2]:
index_creator = VectorstoreIndexCreator()

In [4]:
from langchain.document_loaders import TextLoader
loader = TextLoader('../../state_of_the_union.txt')
docsearch = index_creator.from_loaders([loader]).vectorstore

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


In [6]:
from langchain.chat.vector_db_qa import VectorDBQA
from langchain.chat_models import ChatOpenAI

In [8]:
chain = VectorDBQA.from_model(model = ChatOpenAI(temperature=0), vectorstore=docsearch)

In [10]:
query = "What did the president say about Justice Breyer"
chain.run(query=query)

'The President honored Justice Stephen Breyer, an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court, for his dedicated service to the country. The President also mentioned that one of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court, and he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to continue Justice Breyer’s legacy of excellence.'